# Summary

This notebook showcases a coverage simulation in the Arc de Triomphe area of Paris using Sionna, an open-source library developed by Nvidia for creating digital twins of cellular mobile networks. The simulation focuses on RF optimization, specifically the impact of upgrading antenna array systems from a configuration of 8x2 UPA to 16x16 UPA at 28GHz. The goal is to evaluate improvements in key performance indicators (KPI), such as the cumulative distribution function (CDF) of signal-to-interference-plus-noise ratio (SINR) over received signal strength (RSS). The data used is sourced from OpenStreetMap and visualized with Blender.

## Usecase

**Role name:** RF optimization engineers. 

**Proejct:** 
In telco providers, there are multiple RF planning and optimization teams that plan, implement and test changes in RF network. They track network demand and proactively or based on customer complaints adjust network capacity/config to meet the demands. 

Sometime it means installing new tower, adjusting physical configs permanently or temporality to meet the demands. Change variables in a BSS tower can be: 
- antenna location
- antenna height
- antenna array system
- antenna orientation(tilt and azimuth) 
- Tx power
- Frequency
- Bandwidth 
- Antenna gain 
- ...

Normally they simulate first before affecting outside world, once they have a good understaning of change results, it will be implements in network. This simulation happens in third party software tools that are not open source and have partial view of network. This will reslut in 
- Suboptimal solution
- Expensive siloed processes
- Customer dissatisfaction 
- Increases number of tickets
- reactive process 



In this simulation we use **Sionna** to simulate the effect of upgrading antenna array systems of the network from config 1 to config 2 in 28GHz. 

- **Config 1: transmitter: [8x2 UPA], user: [2x2 UPA]**
- **Config 2: transmitter: [16x16 UPA], user: [2x2 UPA]**

to see if our target KPI (in this case CDF of SINR or RSS) gets better or not. (there are many other KPIs and the root complaint identofies which kpi to improve)

UPA: uniform planar array 




Sionna is an open source library developed by Nvidia. Sionna is Nvidia's flagship software for creating **digital twin** of cellular mobile network. 
This is a very active area of development in Nvidia, they had a releaase in GTC 2025, [last week](https://www.linkedin.com/posts/faycal-ait-aoudia-166320137_6g-wirelesscommunications-sionna-activity-7308044398531956736-Hpj8?utm_source=share&utm_medium=member_desktop&rcm=ACoAABFAqOYBj-sgobPtE8yIpU46ThzZ_zWbPyw )





# Coverage simulation in Arc de Triomphe in Paris


<img src="Screenshot 2025-03-24 at 7.52.38 PM.png" alt="drawing" width="500"/>

Credit Sionna RT: The scene was created with data downloaded from [OpenStreetMap](https://www.openstreetmap.org/) and the help of [Blender](https://www.blender.org/) and the [Blender-OSM](https://github.com/vvoovv/blender-osm) and [Mitsuba Blender](https://github.com/mitsuba-renderer/mitsuba-blender) add-ons. The data is licensed under [the Open Data Commons Open Database License (ODbL)](https://openstreetmap.org/copyright).

This is a simple showcase adopted from [here](https://colab.research.google.com/github/nvlabs/sionna-rt/blob/main/tutorials/Radio-Maps.ipynb#scrollTo=7BEQgaFJbcJ1) which is a notebook released by Nvidia's 6G research team showcasing Sionna's capabilities. 




Our network is consist of: 

- we have 7 cells 
- 50 users per cell
- fixed antenna location (specified below)
- fixed antenna height (specified below)
- fixed antenna power (specified below)
- fixed antenna orientation (specified below)
- fixed bandwidth 100 MHz (specified below)
- fixed carrier frequency 28 GHz (specified below)



 

In [0]:
%python
# Install the required packages
%pip install drjit mitsuba sionna-rt

In [0]:
import numpy as np
import drjit as dr
import mitsuba as mi
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import colormaps

# Import or install Sionna
try:
    import sionna.rt
except ImportError as e:
    import os
    os.system("pip install sionna-rt")
    import sionna.rt

from sionna.rt import LambertianPattern, DirectivePattern, BackscatteringPattern,\
                      load_scene, Camera, Transmitter, Receiver, PlanarArray,\
                      PathSolver, RadioMapSolver, cpx_abs, cpx_convert

no_preview = True # Toggle to False to use the preview widget
                  # instead of rendering for scene visualization


from sionna.rt import load_scene, PlanarArray, Transmitter, Receiver, Camera, watt_to_dbm,\
                      RadioMapSolver, PathSolver

In [0]:
class SceneConfiguration:
  def __init__(self, num_rows_tx, num_cols_tx, num_rows_rx, num_cols_rx):
        self.num_rows_tx = num_rows_tx
        self.num_cols_tx = num_cols_tx
        self.num_rows_rx = num_rows_rx
        self.num_cols_rx = num_cols_rx
        # Place transmitters
        self.positions = np.array( #lat, long, height
                  [[-150.3, 21.63, 42.5],
                    [-125.1, 9.58, 42.5],
                    [-104.5, 54.94, 42.5],
                    [-128.6, 66.73, 42.5],
                    [172.1, 103.7, 24],
                    [232.8, -95.5, 17],
                    [80.1, 193.8, 21]
                  ])
        self.look_ats = np.array( #orientation
                  [[-216, -21,0],
                  [-90, -80, 0],
                  [-16.5, 75.8, 0],
                  [-164, 153.7, 0],
                  [247, 92, 0],
                  [211, -180, 0],
                  [126.3, 194.7, 0]
                  ])
        self.power_dbms = [23, 23, 23, 23, 23, 23, 23] #power

  def config_scene(self):
      """Load and configure a scene"""
      scene = load_scene(sionna.rt.scene.etoile)
      scene.bandwidth=100e6
      scene.frequency = 2.8e9

      # Configure antenna arrays for all transmitters and receivers
      scene.tx_array = PlanarArray(num_rows=self.num_rows_tx,
                                  num_cols=self.num_cols_tx,
                                  pattern="tr38901",
                                  polarization="V")

      scene.rx_array = PlanarArray(num_rows=self.num_rows_rx,
                                  num_cols=self.num_cols_rx,
                                  pattern="tr38901",
                                  polarization="V")



      for i, position in enumerate(self.positions):
          scene.add(Transmitter(name=f'tx{i}',
                                position=position,
                                look_at=self.look_ats[i],
                                power_dbm=self.power_dbms[i]))

      return scene

## Config 1

We have a cell transmitter with [8x2 UPA] and user cellphone with [2x2 UPA]. It means antenna array system of transmitter has 16 components arranged in 8 by 2 rectangular shape, and antenna array system of Cell phone has 4 components arranged in 2 by 2 square shape. 

<img src="./Screenshot 2025-03-24 at 7.37.02 PM.png" alt="Config 1" width="300"/>

In [0]:

rm_solver = RadioMapSolver() #solves maxwell's equations for the configured scene

# Load and configure scene
scene_config1 = SceneConfiguration(num_rows_tx=8, num_cols_tx=2, num_rows_rx=2, num_cols_rx=2)
scene1 = scene_config1.config_scene()
# Compute the SINR map
rm_etoile1 = rm_solver(scene1,
                      max_depth=5,
                      samples_per_tx=10**7,
                      cell_size=(1, 1))

In [0]:
if no_preview:
    # Render an image
    cam = Camera(position=[0,0,1000],
                     orientation=np.array([0,np.pi/2,-np.pi/2]))
    scene1.render(camera=cam,
                        radio_map=rm_etoile1,
                        rm_metric="sinr",
                        rm_vmin=-10,
                        rm_vmax=60, rm_show_color_bar=True);
else:
    # Show preview
    scene1.preview(radio_map=rm_etoile1,
                         rm_metric="sinr",
                         rm_vmin=-10,
                         rm_vmax=70)

With a radio map at hand, we can now sample random positions at which we place receivers and then compute channel impulse responses.

In [0]:
rm_etoile1.show_association("sinr");

pos, cell_ids = rm_etoile1.sample_positions(
          num_pos=50,
          metric="sinr",
          min_val_db=3,
          min_dist=10,
          max_dist=200,
          tx_association=True)

fig = rm_etoile1.show(metric="sinr", vmin = -10, vmax=70);

# One criteria for cell selection is to choose the cell with highest SINR. Here is the plot of highest SINR based on the computed radio map
# Visualize sampled positions
for tx, ids in enumerate(cell_ids.numpy()):
    fig.axes[0].plot(ids[:,1], ids[:,0],
                     marker='o',
                     markersize=2,
                     linestyle='',
                     color=mpl.colormaps['Dark2'].colors[tx])




Cell-to-Tx association plot: shows cell coverage, you can see we have 7 cell coverage in 7 different colors

Users are connected to cells with maximum SINR in that location, which is shown in the seconf plot. 
- brigh green means high SINR (Good)
- dark blue means low SINR (Bad)


 you can see users assigned to each tower in Higherst SINR across all TXs plot

In [0]:
rm_etoile1.cdf(metric="sinr", bins = 400)
plt.xlim(-40, 75)
rm_etoile1.cdf(metric="rss", bins = 400)
plt.xlim(-150, 25)

Visualize CDF of received signal strength (RSS), and signal-to-interference-plus-noise ratio.  We will monitor these two KPI for determining if change in network was overally positive and it improved user experince from perspective of SINR and RSS or not.

## Config 2

We have a cell transmitter with [8x8 UPA] and user cellphone with [2x2 UPA]. It means antenna array system of transmitter has 16 components arranged in 8 by 8 square shape, and antenna array system of Cell phone has 4 components arranged in 2 by 2 square shape. 


<img src="./Screenshot 2025-03-24 at 8.24.49 PM.png" alt="Config 2" width="300"/>


In [0]:

rm_solver = RadioMapSolver() #solves maxwell's equations for the configured scene

# Load and configure scene
scene_config2 = SceneConfiguration(num_rows_tx=8, num_cols_tx=8, num_rows_rx=2, num_cols_rx=2)
scene2 = scene_config2.config_scene()
# Compute the SINR map
rm_etoile2 = rm_solver(scene2,
                      max_depth=5,
                      samples_per_tx=10**7,
                      cell_size=(1, 1))

In [0]:
if no_preview:
    # Render an image
    cam = Camera(position=[0,0,1000],
                     orientation=np.array([0,np.pi/2,-np.pi/2]))
    scene2.render(camera=cam,
                        radio_map=rm_etoile2,
                        rm_metric="sinr",
                        rm_vmin=-10,
                        rm_vmax=60, rm_show_color_bar=True);
else:
    # Show preview
    scene2.preview(radio_map=rm_etoile2,
                         rm_metric="sinr",
                         rm_vmin=-10,
                         rm_vmax=70)

With a radio map at hand, we can now sample random positions at which we place receivers and then compute channel impulse responses.

In [0]:
rm_etoile2.show_association("sinr");

pos, cell_ids = rm_etoile2.sample_positions(
          num_pos=50,
          metric="sinr",
          min_val_db=3,
          min_dist=10,
          max_dist=200,
          tx_association=True)

fig = rm_etoile2.show(metric="sinr", vmin = -10, vmax=70);

# One criteria for cell selection is to choose the cell with highest SINR. Here is the plot of highest SINR based on the computed radio map
# Visualize sampled positions
for tx, ids in enumerate(cell_ids.numpy()):
    fig.axes[0].plot(ids[:,1], ids[:,0],
                     marker='o',
                     markersize=2,
                     linestyle='',
                     color=mpl.colormaps['Dark2'].colors[tx])




Cell-to-Tx association plot: shows cell coverage, you can see we have 7 cell coverage in 7 different colors

Users are connected to cells with maximum SINR in that location, which is shown in the seconf plot. 
- brigh green means high SINR (Good)
- dark blue means low SINR (Bad)


 you can see users assigned to each tower in Higherst SINR across all TXs plot

In [0]:
rm_etoile2.cdf(metric="sinr", bins = 400)
plt.xlim(-40, 75)
rm_etoile2.cdf(metric="rss", bins = 400)
plt.xlim(-150, 25)

Visualize CDF of received signal strength (RSS), and signal-to-interference-plus-noise ratio. 




# Conclusion 
# 
with siumulating these two config, RF optimizer can see that user experience has decreases. (CDF moved to left) thereffore it is not a good change. 


![Screenshot 2025-03-28 at 1.13.38 PM.png](./Screenshot 2025-03-28 at 1.13.38 PM.png "Screenshot 2025-03-28 at 1.13.38 PM.png")

 you can find more infomation on rate racing module here: https://nvlabs.github.io/sionna/rt/tutorials.html 